In [38]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf

# --- 1. ADATOK BETÖLTÉSE ---
print("Adatok betöltése...")
try:
    # Feltételezzük, hogy a stock_market_final_panel.csv-ben a PAST már a 20 napos átlag
    df = pd.read_csv("stock_market_final_panel.csv")
    df['Date'] = pd.to_datetime(df['Date'])
    print(f"Sikeres betöltés: {len(df)} sor.")
except FileNotFoundError:
    print("HIBA: Nem találom a 'stock_market_final_panel.csv' fájlt!")
    exit()

# --- 2. BENCHMARK (ACWI) ELŐKÉSZÍTÉSE ---
print("Benchmark (ACWI) letöltése...")
try:
    acwi = yf.download("ACWI", start="2021-06-01", end="2022-04-01", progress=False)
    if not acwi.empty:
        # Árfolyam kinyerése (Close)
        price = acwi['Close'].iloc[:, 0] if (isinstance(acwi.columns, pd.MultiIndex) and 'Close' in acwi.columns) else acwi['Close'] if 'Close' in acwi.columns else acwi.iloc[:, 0]
        if isinstance(price, pd.DataFrame): price = price.squeeze()
        
        # Loghozam számítása
        market_return = np.log(price / price.shift(1)).rename("Market_Return")
        print("✅ Benchmark OK.")
    else:
        raise ValueError("Üres adat")
except:
    print("⚠️ Benchmark letöltés sikertelen, országok átlagát használjuk proxyként.")
    market_return = df.groupby('Date')['Return'].mean().rename("Market_Return")

# --- 3. CAR SZÁMÍTÁSA (Event Study) ---
EVENT_DATE = pd.Timestamp("2022-02-24")
results = []
countries = df['Country'].unique()

print(f"CAR számítása {len(countries)} országra...")

for country in countries:
    try:
        # Adatok szűrése az adott országra
        c_data = df[df['Country'] == country].set_index('Date').sort_index()
        
        # Összefésülés a Benchmarkkal
        merged = pd.concat([c_data['Return'], market_return], axis=1).dropna()
        
        if EVENT_DATE not in merged.index: continue
        loc = merged.index.get_loc(EVENT_DATE)
        
        # Becslési ablak: -150 naptól -6 napig (a te kódod szerint loc-140)
        est = merged.iloc[loc-140 : loc-6]
        if len(est) < 30: continue
        
        # Market Model becslése (Alpha, Beta)
        model = sm.OLS(est['Return'], sm.add_constant(est['Market_Return'])).fit()
        alpha, beta = model.params['const'], model.params['Market_Return']
        
        # CAR számítása [+1, +3] ablakra
        # Az ablak indexei: loc+1 -től loc+3-ig (a szeletelés vége nem inkluzív, ezért +4)
        win_data = merged.iloc[loc+1 : loc+4]
        
        expected_ret = alpha + beta * win_data['Market_Return']
        abnormal_ret = win_data['Return'] - expected_ret
        car = abnormal_ret.sum() * 100 # Százalékban
        
        # Metaadatok a regresszióhoz (az első sorból vesszük ki a fix értékeket)
        meta = c_data.iloc[0]
        
        results.append({
            'Country': country,
            'CAR': car,
            'NATO': meta['NATO'],
            'DEV': meta['DEV'],
            'TGDP': meta['TRADE'],   # A képen ez TGDP
            'PAST': meta['PAST'] * 100 if pd.notnull(meta['PAST']) else 0, # Sárgával jelölt változó
            'EXRATE': meta['EXRATE']
        })
    except Exception as e:
        pass

# --- 4. KERESZTMETSZETI REGRESSZIÓ (Cross-Sectional Regression) ---
# Ez felel meg a képen látható egyenletnek
reg_df = pd.DataFrame(results).dropna()

# --- EXRATE JAVÍTÁS (Invertálás) ---
# Itt végezzük el a cserét: Local/USD -> USD/Local (hogy a skálák helyesek legyenek)
if not reg_df.empty:
    reg_df['EXRATE'] = 1 / reg_df['EXRATE']

print("\n" + "="*60)
print("TABLE 5 REPRODUKCIÓ (A feltöltött képlet alapján)")
print("Equation: CAR = α + β1*NATO + β2*DEV + β3*TGDP + β4*PAST + β5*EXRATE")
print("="*60)

if len(reg_df) > 10:
    # Független változók (X)
    X_vars = ['NATO', 'DEV', 'TGDP', 'PAST', 'EXRATE']
    X = reg_df[X_vars]
    X = sm.add_constant(X) # Konstans (α) hozzáadása
    
    # Függő változó (Y)
    Y = reg_df['CAR']
    
    # Modell futtatása (Robust Standard Errors - HC1)
    model_reg = sm.OLS(Y, X).fit(cov_type='HC1')
    
    print(model_reg.summary())
    
    # Értékelés
    print("\n--- Eredményelemzés ---")
    nato_p = model_reg.pvalues['NATO']
    if nato_p < 0.05:
        print(f"✅ A NATO hatás szignifikáns! (p={nato_p:.4f})")
    else:
        print(f"❌ A NATO hatás nem szignifikáns. (p={nato_p:.4f})")
else:
    print("HIBA: Túl kevés adat áll rendelkezésre a regresszióhoz.")

Adatok betöltése...
Sikeres betöltés: 33690 sor.
Benchmark (ACWI) letöltése...


C:\Users\andre\AppData\Local\Temp\ipykernel_27992\4093001565.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  acwi = yf.download("ACWI", start="2021-06-01", end="2022-04-01", progress=False)


✅ Benchmark OK.
CAR számítása 46 országra...

TABLE 5 REPRODUKCIÓ (A feltöltött képlet alapján)
Equation: CAR = α + β1*NATO + β2*DEV + β3*TGDP + β4*PAST + β5*EXRATE
                            OLS Regression Results                            
Dep. Variable:                    CAR   R-squared:                       0.285
Model:                            OLS   Adj. R-squared:                  0.191
Method:                 Least Squares   F-statistic:                     2.710
Date:                Thu, 18 Dec 2025   Prob (F-statistic):             0.0344
Time:                        15:56:57   Log-Likelihood:                -104.51
No. Observations:                  44   AIC:                             221.0
Df Residuals:                      38   BIC:                             231.7
Df Model:                           5                                         
Covariance Type:                  HC1                                         
                 coef    std err          z  

In [39]:
# --- 1. BEÁLLÍTÁSOK ÉS ADATOK BETÖLTÉSE ---
EVENT_DATE = pd.Timestamp("2022-02-24")
# A cikk szerinti ablakok (Table 5)
WINDOWS = [(-5, -1), (-3, -1), (0, 0), (1, 3), (1, 5)]
WINDOW_NAMES = ['[-5,-1]', '[-3,-1]', '[0, 0]', '[+1,+3]', '[+1,+5]']

print("--- 1. LÉPÉS: ADATOK BETÖLTÉSE ---")
try:
    df = pd.read_csv("stock_market_final_panel.csv")
    df['Date'] = pd.to_datetime(df['Date'])
    print(f"✅ Adatbázis betöltve: {len(df)} sor, {df['Country'].nunique()} ország.")
except FileNotFoundError:
    print("❌ HIBA: Nem találom a 'stock_market_final_panel.csv' fájlt.")
    raise

# --- 2. LÉPÉS: BENCHMARK (ACWI) LETÖLTÉSE ---
print("\n--- 2. LÉPÉS: BENCHMARK (ACWI) LETÖLTÉSE ---")
try:
    acwi = yf.download("ACWI", start="2021-06-01", end="2022-04-01", progress=False)
    
    if not acwi.empty:
        # Árfolyam kinyerése
        if isinstance(acwi.columns, pd.MultiIndex):
            price = acwi['Close'].iloc[:, 0]
        elif 'Close' in acwi.columns:
            price = acwi['Close']
        else:
            price = acwi.iloc[:, 0]
            
        market_return = np.log(price / price.shift(1)).rename("Market_Return")
        print("✅ Benchmark (ACWI) sikeresen letöltve.")
    else:
        raise ValueError("Üres adat.")

except Exception as e:
    print(f"⚠️ Hiba a benchmarknál ({e}), átlagot használunk.")
    market_return = df.groupby('Date')['Return'].mean().rename("Market_Return")

# --- 3. LÉPÉS: CAR SZÁMÍTÁS (EZT HIÁNYOLTA A GÉP!) ---
print("\n--- 3. LÉPÉS: CAR SZÁMÍTÁS (Market Model) ---")
results = []
countries = df['Country'].unique()

for country in countries:
    try:
        c_data = df[df['Country'] == country].set_index('Date').sort_index()
        if c_data.empty or c_data['Return'].isnull().all(): continue

        merged = pd.concat([c_data['Return'], market_return], axis=1).dropna()
        if EVENT_DATE not in merged.index: continue
            
        loc = merged.index.get_loc(EVENT_DATE)
        
        # Becslési ablak: [-140, -6]
        est_start, est_end = loc - 140, loc - 6
        if est_start < 0: continue
        
        est_data = merged.iloc[est_start : est_end]
        if len(est_data) < 30: continue
        
        # Market Model
        model = sm.OLS(est_data['Return'], sm.add_constant(est_data['Market_Return'])).fit()
        alpha, beta = model.params['const'], model.params['Market_Return']
        
        # Metaadatok
        base_row = c_data.iloc[0]
        row_dict = {
            'Country': country,
            'NATO': base_row['NATO'],
            'DEV': base_row['DEV'],
            'TGDP': base_row['TRADE'],
            'EXRATE': base_row['EXRATE'],
            'PAST': base_row['PAST']
        }
        
        # CAR számítása ablakokra
        for start_day, end_day in WINDOWS:
            win_data = merged.iloc[loc + start_day : loc + end_day + 1]
            expected = alpha + beta * win_data['Market_Return']
            abnormal = win_data['Return'] - expected
            # Itt jönnek létre a 'CAR_x_y' oszlopok!
            row_dict[f"CAR_{start_day}_{end_day}"] = abnormal.sum() * 100
            
        results.append(row_dict)
    except Exception:
        continue

# Itt hozzuk létre a reg_df-et, ami már tartalmazza a CAR oszlopokat
reg_df = pd.DataFrame(results)
print(f"✅ CAR számítás kész. Feldolgozott országok: {len(reg_df)}")

# --- 4. LÉPÉS: ADATOK ELŐKÉSZÍTÉSE ÉS TISZTÍTÁSA ---
# Árfolyam javítása: Reciprok képzése, de figyelünk a 0-val való osztásra
reg_df['EXRATE_FIXED'] = reg_df['EXRATE'].apply(lambda x: 1/x if (pd.notnull(x) and x != 0) else np.nan)
reg_df['EXRATE']=reg_df['EXRATE_FIXED']

# Független változók listája
X_vars = ['NATO', 'DEV', 'TGDP', 'PAST', 'EXRATE']

# ⚠️ FONTOS: Tisztítás - eldobjuk a sorokat, ahol NaN vagy Inf van
# Először a végteleneket (inf) NaN-ra cseréljük
reg_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Majd minden sort kidobunk, ahol bármelyik változó hiányzik
reg_df_clean = reg_df.dropna(subset=X_vars + [f"CAR_{start}_{end}" for start, end in WINDOWS])

print(f"Eredeti országok száma: {len(reg_df)}")
print(f"Tisztított minta mérete: {len(reg_df_clean)} (Ezeken fut a regresszió)")

if len(reg_df) != len(reg_df_clean):
    print("Kiesett országok (hiányzó adat miatt):", set(reg_df['Country']) - set(reg_df_clean['Country']))

# --- 5. LÉPÉS: TÁBLÁZAT GENERÁLÁSA (TABLE 5) ---
print("\n" + "="*75)
print("TABLE 5 REPRODUKCIÓ: Cross-sectional regression analysis (Cleaned Data)")
print("="*75)

# X mátrix (Konstanssal) a TISZTÍTOTT adaton
X = sm.add_constant(reg_df_clean[X_vars])

# Fejléc
header = f"{'Variable':<15}"
for name in WINDOW_NAMES:
    header += f"{name:>12}"
print(header)
print("-" * len(header))

# Adatok tárolása
output_rows = {var: [] for var in X_vars} 
tstat_rows = {var: [] for var in X_vars}  
stats_rows = {'Adj. R2': [], 'F-stat': [], 'Obs': []}

for start, end in WINDOWS:
    y_col = f"CAR_{start}_{end}"
    Y = reg_df_clean[y_col]
    
    # OLS futtatása
    try:
        model = sm.OLS(Y, X).fit(cov_type='HC1')
        
        # Statisztikák mentése
        stats_rows['Adj. R2'].append(f"{model.rsquared_adj:.3f}")
        f_stars = "***" if model.f_pvalue < 0.01 else "**" if model.f_pvalue < 0.05 else "*" if model.f_pvalue < 0.1 else ""
        stats_rows['F-stat'].append(f"{model.fvalue:.2f}{f_stars}")
        stats_rows['Obs'].append(str(int(model.nobs)))
        
        # Változók mentése
        for var in X_vars:
            coef = model.params[var]
            t_val = model.tvalues[var]
            p_val = model.pvalues[var]
            stars = "***" if p_val < 0.01 else "**" if p_val < 0.05 else "*" if p_val < 0.1 else ""
            
            output_rows[var].append(f"{coef:.4f}{stars}")
            tstat_rows[var].append(f"({t_val:.2f})")
            
    except Exception as e:
        # Ha a regresszió elszáll (pl. szinguláris mátrix), jelezzük
        for var in X_vars:
            output_rows[var].append("Error")
            tstat_rows[var].append("(-)")
        stats_rows['Adj. R2'].append("-")
        stats_rows['F-stat'].append("-")
        stats_rows['Obs'].append("0")

# Táblázat kirajzolása
for var in X_vars:
    row_str = f"{var:<15}"
    for val in output_rows[var]:
        row_str += f"{val:>12}"
    print(row_str)
    
    t_str = f"{'':<15}"
    for val in tstat_rows[var]:
        t_str += f"{val:>12}"
    print(t_str)

print("-" * len(header))

for stat_name in ['Adj. R2', 'F-stat', 'Obs']:
    row_str = f"{stat_name:<15}"
    for val in stats_rows[stat_name]:
        row_str += f"{val:>12}"
    print(row_str)

print("="*75)
print("Jelmagyarázat: *** p<0.01, ** p<0.05, * p<0.1")

--- 1. LÉPÉS: ADATOK BETÖLTÉSE ---
✅ Adatbázis betöltve: 33690 sor, 46 ország.

--- 2. LÉPÉS: BENCHMARK (ACWI) LETÖLTÉSE ---


C:\Users\andre\AppData\Local\Temp\ipykernel_27992\789871811.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  acwi = yf.download("ACWI", start="2021-06-01", end="2022-04-01", progress=False)


✅ Benchmark (ACWI) sikeresen letöltve.

--- 3. LÉPÉS: CAR SZÁMÍTÁS (Market Model) ---
✅ CAR számítás kész. Feldolgozott országok: 45
Eredeti országok száma: 45
Tisztított minta mérete: 44 (Ezeken fut a regresszió)
Kiesett országok (hiányzó adat miatt): {'Taiwan'}

TABLE 5 REPRODUKCIÓ: Cross-sectional regression analysis (Cleaned Data)
Variable            [-5,-1]     [-3,-1]      [0, 0]     [+1,+3]     [+1,+5]
---------------------------------------------------------------------------
NATO                -0.6655      0.2416     -1.2866     -0.9516     -0.4482
                    (-1.14)      (0.64)     (-1.37)     (-0.73)     (-0.33)
DEV                  0.0367     -0.0659      0.6890      1.3171     -0.7584
                     (0.07)     (-0.17)      (0.84)      (0.93)     (-0.62)
TGDP               -0.0057*  -0.0058***    -0.0069*     -0.0059     -0.0071
                    (-1.75)     (-3.44)     (-1.82)     (-1.09)     (-1.52)
PAST              287.4416*  237.7680**    396.9517  83

In [40]:
# --- MINTA ELLENŐRZÉSE ---

# 1. A regresszióba bekerült országok lekérése
# (A kódodban 'reg_df' vagy 'res_df' a neve a tisztított táblának)
if 'reg_df' in locals():
    final_data = reg_df
elif 'res_df' in locals():
    final_data = res_df
else:
    print("Hiba: Nem találom a regressziós táblát. Futtasd le előbb a regressziót!")
    final_data = pd.DataFrame()

if not final_data.empty:
    included_list = final_data['Country'].unique()
    included_list.sort() # Ábécé sorrendbe rendezzük
    
    print("="*50)
    print(f"A MODELLBE BEKERÜLT ORSZÁGOK LISTÁJA (N={len(included_list)})")
    print("="*50)
    
    for i, country in enumerate(included_list, 1):
        print(f"{i}. {country}")
        
    # 2. Megnézzük, kik maradtak ki
    all_countries = df['Country'].unique()
    excluded_list = list(set(all_countries) - set(included_list))
    
    if excluded_list:
        print("\n" + "-"*50)
        print(f"KIMARADT ({len(excluded_list)} db):")
        for c in excluded_list:
            print(f"❌ {c}")
    else:
        print("\n✅ Minden ország bekerült!")

A MODELLBE BEKERÜLT ORSZÁGOK LISTÁJA (N=45)
1. Australia
2. Austria
3. Belgium
4. Brazil
5. Canada
6. Chile
7. China
8. Colombia
9. Czech Republic
10. Egypt
11. Finland
12. France
13. Germany
14. Greece
15. Hong Kong
16. Hungary
17. India
18. Indonesia
19. Ireland
20. Israel
21. Italy
22. Japan
23. Kuwait
24. Malaysia
25. Mexico
26. Netherlands
27. New Zealand
28. Norway
29. Peru
30. Philippines
31. Portugal
32. Qatar
33. Saudi Arabia
34. Singapore
35. South Africa
36. South Korea
37. Spain
38. Sweden
39. Switzerland
40. Taiwan
41. Thailand
42. Turkey
43. UAE
44. UK
45. US

--------------------------------------------------
KIMARADT (1 db):
❌ Denmark
